In [1]:
import os
import random
import statsmodels.api as sm
import pylab as pl
import pandas as pd
import numpy as np
#import xgboost as xgb
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor, RandomForestClassifier
from sklearn import metrics, svm, preprocessing
from pandas import DataFrame
from keras import backend as K
import sklearn
from sklearn.model_selection import train_test_split, KFold
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Input, BatchNormalization
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from IPython.display import SVG
from keras.utils import plot_model
import time
from sklearn.manifold import TSNE

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import gc
import pickle

C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  stacklevel=1)
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [2]:
help(LogisticRegression)

Help on class LogisticRegression in module sklearn.linear_model._logistic:

class LogisticRegression(sklearn.linear_model._base.LinearClassifierMixin, sklearn.linear_model._base.SparseCoefMixin, sklearn.base.BaseEstimator)
 |  LogisticRegression(penalty='l2', *, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)
 |  
 |  Logistic Regression (aka logit, MaxEnt) classifier.
 |  
 |  In the multiclass case, the training algorithm uses the one-vs-rest (OvR)
 |  scheme if the 'multi_class' option is set to 'ovr', and uses the
 |  cross-entropy loss if the 'multi_class' option is set to 'multinomial'.
 |  (Currently the 'multinomial' option is supported only by the 'lbfgs',
 |  'sag', 'saga' and 'newton-cg' solvers.)
 |  
 |  This class implements regularized logistic regression using the
 |  'liblinear' library, 'newton-cg', 's

In [2]:
def load_data(root = '', MetaFile = 'ADoub_meta.csv', ADoubFile = 'ADoub.npy'):

    ADoub_meta = pd.read_csv(root + MetaFile)
    X = np.load(root + ADoubFile)
    
    return X, ADoub_meta

def gen_label(Y_raw):
    
    Y_raw = Y_raw.astype(int)
    Y = np.zeros((Y_raw.shape[0], Y_raw.max() + 1))
    for i, idx in enumerate(Y_raw):
        Y[i, idx] = 1
        
    return Y

def FCNN(input_shape, out_shape, nodes = [2048, 1024, 512]):
    
    X_input = Input(input_shape)
    
    X = X_input
    
    for node in nodes:
        X = Dense(node, init = 'glorot_normal', activation = 'relu')(X)
        X = BatchNormalization(axis = 1)(X)
    
    X = Dense(out_shape, init = 'glorot_normal', activation = 'softmax')(X)
    
    model = Model(inputs = X_input, outputs = X, name = 'FCNN')
    
    return model

def batch_generator(X, y, sample_index, batch_size, shuffle):
    number_of_batches = int(np.ceil(len(sample_index)/batch_size))
    counter = 0

    if shuffle:
        np.random.shuffle(sample_index)
    while True:
        batch_index = sample_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[batch_index, :]
        y_batch = y[batch_index, :]
        counter += 1
        yield X_batch, y_batch
        if (counter == number_of_batches):
            if shuffle:
                np.random.shuffle(sample_index)
            counter = 0
            
def CrossValidation(X, Y, fold = 5, CV = True, epoch = 20, batch_size = 4096, nodes = [2048, 1024, 512]):

    kf = KFold(n_splits=fold, random_state = 0, shuffle = True)
    cv_index = kf.split(X)

    length = Y.shape[1]

    earlystopper = EarlyStopping(patience=5, verbose=1)

    # Set a learning rate annealer
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                                patience=3, 
                                                verbose=1, 
                                                factor=0.5, 
                                                min_lr=0.00001)
    
    histories = []

    i = 0

    for cv_train, cv_test in cv_index:

        if((not CV) & (i!= 0)):
            break
        i += 1
        print(i)

        X_valid = X[cv_test, :].copy()

        model = FCNN((X.shape[1],), length, nodes = nodes)
        model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

        history = model.fit_generator(generator = batch_generator(X,\
                                                        Y,\
                                                        cv_train,\
                                                        batch_size, True),\
                            epochs = epoch,\
                            steps_per_epoch = len(cv_train) / batch_size,\
                            validation_data = (X_valid, Y[cv_test, :]),\
                            callbacks = [learning_rate_reduction, earlystopper])
        
        histories.append(history)

        gc.collect()
    if CV:
        return histories
    else:
        return history

def Train(X, Y, root, name = '1_9_1', epoch = 20, batch_size = 4096, nodes = [2048, 1024, 512]):
    
    length = Y.shape[1]

    earlystopper = EarlyStopping(patience=5, verbose=1)

    # Set a learning rate annealer
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                                patience=3, 
                                                verbose=1, 
                                                factor=0.5, 
                                                min_lr=0.00001)

    model = FCNN((X.shape[1],), length, nodes = nodes)
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

    model.fit_generator(generator = batch_generator(X,\
                                                    Y,\
                                                    np.arange(X.shape[0]),\
                                                    batch_size, True),\
                        epochs = epoch,\
                        steps_per_epoch = X.shape[0] / batch_size,\
                        callbacks = [learning_rate_reduction, earlystopper])

    model.save(root + 'Decomp_' + name + '.h5')
    
def SaveValidation(history, CV, name):
    
    if(CV):
        histories = history
        
        with open(save_root + 'training_log_' + name + '.pickle', 'wb') as handle:
            pickle.dump([history.history for history in histories], handle)

        with open(save_root + 'training_log_' + name + '.pickle', 'rb') as handle:
            histories = pickle.load(handle)

        accuracy = []

        for history in histories:
            accuracy.append(history['val_accuracy'][-1])

        print('Validation Accuracy')
        print(accuracy)
        print(np.mean(accuracy))
        
    else:
        with open(save_root + 'training_log_' + name + '.pickle', 'wb') as handle:
            pickle.dump(history, handle)

        with open(save_root + 'training_log_' + name + '.pickle', 'rb') as handle:
            history = pickle.load(handle)
            
        print(history.history['val_accuracy'][-1])
        
def CheckAccuracy(name):
    with open(save_root + 'training_log_' + name + '.pickle', 'rb') as handle:
        histories = pickle.load(handle)

        accuracy = []

        for history in histories:
            accuracy.append(history['val_accuracy'][-1])

        print(name)
        print('Validation Accuracy')
        print(accuracy)
        print(np.mean(accuracy))

In [8]:
file_root = 'ADoub_new/'
save_root = 'DecompModel/'

if not os.path.exists(save_root):
    os.makedirs(save_root)
    
X, ADoub_meta = load_data(file_root, 'ADoub_meta.csv', 'ADoub.npy')

Y1 = gen_label(ADoub_meta.iloc[:, 0])
Y2 = gen_label(ADoub_meta.iloc[:, 2])

print(Y1.shape)
print(Y2.shape)
print(X.shape)
print(X.sum(axis = 1))

(400000, 1415)
(400000, 1203)
(400000, 8676)
[1000000. 1000000. 1000000. ... 1000000. 1000000. 1000000.]


# FCNN

In [4]:
#CrossValidation
main_name = 'hep_LEC'
##1
print('*' * 10)
print(1)
print('*' * 10)

name = main_name + '_1'

CV = True

history = CrossValidation(X, Y1, fold = 5, CV=CV, epoch = 20, batch_size = 4096, nodes = [2048, 1024, 512])

SaveValidation(history, CV, name)

**********
1
**********
1
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2048, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1024, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(512, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1415, activation="softmax", kernel_initializer="glorot_normal")`



Epoch 1/20
79/78 [==============================] - 47s 594ms/step - loss: 0.6698 - accuracy: 0.9398 - val_loss: 4.8046 - val_accuracy: 0.6193
Epoch 2/20
79/78 [==============================] - 45s 564ms/step - loss: 0.0344 - accuracy: 0.9942 - val_loss: 1.8805 - val_accuracy: 0.9874
Epoch 3/20
79/78 [==============================] - 45s 563ms/step - loss: 0.0250 - accuracy: 0.9955 - val_loss: 0.2278 - val_accuracy: 0.9924
Epoch 4/20
79/78 [==============================] - 45s 571ms/step - loss: 0.0217 - accuracy: 0.9960 - val_loss: 0.0706 - val_accuracy: 0.9933
Epoch 5/20
79/78 [==============================] - 45s 572ms/step - loss: 0.0175 - accuracy: 0.9966 - val_loss: 0.1034 - val_accuracy: 0.9937
Epoch 6/20
79/78 [==============================] - 45s 574ms/step - loss: 0.0145 - accuracy: 0.9971 - val_loss: 0.0634 - val_accuracy: 0.9943
Epoch 7/20
79/78 [==============================] - 45s 567ms/step - loss: 0.0126 - accuracy: 0.9976 - val_loss: 0.0448 - val_accuracy: 0.994

C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2048, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1024, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(512, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1415, activation="softmax", kernel_initializer="glorot_normal")`


Epoch 1/20
79/78 [==============================] - 47s 594ms/step - loss: 0.6651 - accuracy: 0.9398 - val_loss: 5.0256 - val_accuracy: 0.4978
Epoch 2/20
79/78 [==============================] - 45s 565ms/step - loss: 0.0363 - accuracy: 0.9940 - val_loss: 1.4088 - val_accuracy: 0.9880
Epoch 3/20
79/78 [==============================] - 45s 572ms/step - loss: 0.0249 - accuracy: 0.9955 - val_loss: 0.1581 - val_accuracy: 0.9919
Epoch 4/20
79/78 [==============================] - 45s 569ms/step - loss: 0.0210 - accuracy: 0.9961 - val_loss: 0.0547 - val_accuracy: 0.9932
Epoch 5/20
79/78 [==============================] - 45s 570ms/step - loss: 0.0184 - accuracy: 0.9966 - val_loss: 0.0479 - val_accuracy: 0.9936
Epoch 6/20
79/78 [==============================] - 45s 574ms/step - loss: 0.0150 - accuracy: 0.9972 - val_loss: 0.0509 - val_accuracy: 0.9946
Epoch 7/20
79/78 [==============================] - 47s 592ms/step - loss: 0.0131 - accuracy: 0.9976 - val_loss: 0.0434 - val_accuracy: 0.9945

...

Epoch 1/20
79/78 [==============================] - 47s 596ms/step - loss: 0.6627 - accuracy: 0.9405 - val_loss: 4.9751 - val_accuracy: 0.5627
Epoch 2/20
79/78 [==============================] - 45s 569ms/step - loss: 0.0351 - accuracy: 0.9941 - val_loss: 1.1062 - val_accuracy: 0.9892
Epoch 3/20
79/78 [==============================] - 45s 570ms/step - loss: 0.0255 - accuracy: 0.9955 - val_loss: 0.1732 - val_accuracy: 0.9938
Epoch 4/20
79/78 [==============================] - 45s 571ms/step - loss: 0.0203 - accuracy: 0.9962 - val_loss: 0.0622 - val_accuracy: 0.9943
Epoch 5/20
79/78 [==============================] - 45s 576ms/step - loss: 0.0166 - accuracy: 0.9969 - val_loss: 0.0443 - val_accuracy: 0.9952
Epoch 6/20
79/78 [==============================] - 45s 574ms/step - loss: 0.0156 - accuracy: 0.9970 - val_loss: 0.0400 - val_accuracy: 0.9952
Epoch 7/20
79/78 [==============================] - 46s 581ms/step - loss: 0.0133 - accuracy: 0.9974 - val_loss: 0.0312 - val_accuracy: 0.9953

C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2048, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1024, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(512, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1415, activation="softmax", kernel_initializer="glorot_normal")`


Epoch 1/20
79/78 [==============================] - 47s 594ms/step - loss: 0.6649 - accuracy: 0.9401 - val_loss: 5.0228 - val_accuracy: 0.4560
Epoch 2/20
79/78 [==============================] - 45s 564ms/step - loss: 0.0368 - accuracy: 0.9940 - val_loss: 1.5199 - val_accuracy: 0.9863
Epoch 3/20
79/78 [==============================] - 45s 569ms/step - loss: 0.0259 - accuracy: 0.9957 - val_loss: 0.1695 - val_accuracy: 0.9932
Epoch 4/20
79/78 [==============================] - 45s 568ms/step - loss: 0.0212 - accuracy: 0.9963 - val_loss: 0.0754 - val_accuracy: 0.9933
Epoch 5/20
79/78 [==============================] - 45s 574ms/step - loss: 0.0177 - accuracy: 0.9968 - val_loss: 0.0892 - val_accuracy: 0.9945
Epoch 6/20
79/78 [==============================] - 45s 566ms/step - loss: 0.0159 - accuracy: 0.9971 - val_loss: 0.0804 - val_accuracy: 0.9945
Epoch 7/20
79/78 [==============================] - 45s 571ms/step - loss: 0.0137 - accuracy: 0.9972 - val_loss: 0.0368 - val_accuracy: 0.9951

C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2048, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1024, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(512, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1415, activation="softmax", kernel_initializer="glorot_normal")`


Epoch 1/20
79/78 [==============================] - 46s 582ms/step - loss: 0.6669 - accuracy: 0.9397 - val_loss: 5.0842 - val_accuracy: 0.5830
Epoch 2/20
79/78 [==============================] - 45s 564ms/step - loss: 0.0367 - accuracy: 0.9940 - val_loss: 1.2119 - val_accuracy: 0.9897
Epoch 3/20
79/78 [==============================] - 45s 571ms/step - loss: 0.0255 - accuracy: 0.9954 - val_loss: 0.2275 - val_accuracy: 0.9930
Epoch 4/20
79/78 [==============================] - 45s 571ms/step - loss: 0.0217 - accuracy: 0.9960 - val_loss: 0.0690 - val_accuracy: 0.9938
Epoch 5/20
79/78 [==============================] - 45s 569ms/step - loss: 0.0162 - accuracy: 0.9969 - val_loss: 0.0446 - val_accuracy: 0.9939
Epoch 6/20
79/78 [==============================] - 45s 566ms/step - loss: 0.0142 - accuracy: 0.9971 - val_loss: 0.0478 - val_accuracy: 0.9944
Epoch 7/20
79/78 [==============================] - 44s 561ms/step - loss: 0.0126 - accuracy: 0.9975 - val_loss: 0.0562 - val_accuracy: 0.9948

In [5]:
name = main_name + '_1'
CheckAccuracy(name)

hep_LEC_1
Validation Accuracy
[0.9977874755859375, 0.9976999759674072, 0.998324990272522, 0.99795001745224, 0.9965375065803528]
0.9976599931716919


In [4]:
##2
main_name = 'hep_LEC'
print('*' * 10)
print(2)
print('*' * 10)

name = main_name + '_2'

CV=True

history = CrossValidation(X, Y2, fold = 5, CV=CV, epoch = 20, batch_size = 4096, nodes = [2048, 1024, 512])

SaveValidation(history, CV, name)

**********
2
**********
1
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2048, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1024, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(512, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1203, activation="softmax", kernel_initializer="glorot_normal")`



Epoch 1/20
79/78 [==============================] - 46s 585ms/step - loss: 2.8733 - accuracy: 0.6393 - val_loss: 6.7572 - val_accuracy: 0.0600
Epoch 2/20
79/78 [==============================] - 44s 561ms/step - loss: 0.8640 - accuracy: 0.8675 - val_loss: 5.3085 - val_accuracy: 0.3116
Epoch 3/20
79/78 [==============================] - 44s 557ms/step - loss: 0.5825 - accuracy: 0.8998 - val_loss: 2.2889 - val_accuracy: 0.7111
Epoch 4/20
79/78 [==============================] - 45s 566ms/step - loss: 0.4471 - accuracy: 0.9177 - val_loss: 1.5732 - val_accuracy: 0.7781
Epoch 5/20
79/78 [==============================] - 44s 553ms/step - loss: 0.3718 - accuracy: 0.9289 - val_loss: 1.5119 - val_accuracy: 0.7913
Epoch 6/20
79/78 [==============================] - 44s 557ms/step - loss: 0.3247 - accuracy: 0.9359 - val_loss: 1.6230 - val_accuracy: 0.7926
Epoch 7/20
79/78 [==============================] - 47s 599ms/step - loss: 0.2753 - accuracy: 0.9443 - val_loss: 1.8217 - val_accuracy: 0.767

C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2048, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1024, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(512, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1203, activation="softmax", kernel_initializer="glorot_normal")`


Epoch 1/20
79/78 [==============================] - 47s 589ms/step - loss: 2.8774 - accuracy: 0.6394 - val_loss: 6.7632 - val_accuracy: 0.0600
Epoch 2/20
79/78 [==============================] - 45s 569ms/step - loss: 0.8745 - accuracy: 0.8658 - val_loss: 5.2995 - val_accuracy: 0.3029
Epoch 3/20
79/78 [==============================] - 45s 564ms/step - loss: 0.5926 - accuracy: 0.8978 - val_loss: 2.6478 - val_accuracy: 0.6723
Epoch 4/20
79/78 [==============================] - 46s 577ms/step - loss: 0.4449 - accuracy: 0.9180 - val_loss: 1.8489 - val_accuracy: 0.7505
Epoch 5/20
79/78 [==============================] - 45s 565ms/step - loss: 0.3581 - accuracy: 0.9303 - val_loss: 1.8886 - val_accuracy: 0.7466
Epoch 6/20
79/78 [==============================] - 44s 563ms/step - loss: 0.3136 - accuracy: 0.9384 - val_loss: 1.6017 - val_accuracy: 0.7736
Epoch 7/20
79/78 [==============================] - 44s 563ms/step - loss: 0.2755 - accuracy: 0.9445 - val_loss: 1.5502 - val_accuracy: 0.7931

...

Epoch 1/20
79/78 [==============================] - 46s 577ms/step - loss: 2.8858 - accuracy: 0.6388 - val_loss: 6.7587 - val_accuracy: 0.0496
Epoch 2/20
79/78 [==============================] - 44s 560ms/step - loss: 0.8646 - accuracy: 0.8669 - val_loss: 5.3934 - val_accuracy: 0.2897
Epoch 3/20
79/78 [==============================] - 44s 551ms/step - loss: 0.5862 - accuracy: 0.8981 - val_loss: 2.0673 - val_accuracy: 0.7229
Epoch 4/20
79/78 [==============================] - 44s 554ms/step - loss: 0.4478 - accuracy: 0.9175 - val_loss: 1.5893 - val_accuracy: 0.7688
Epoch 5/20
79/78 [==============================] - 44s 563ms/step - loss: 0.3669 - accuracy: 0.9298 - val_loss: 1.6790 - val_accuracy: 0.7754
Epoch 6/20
79/78 [==============================] - 44s 555ms/step - loss: 0.3268 - accuracy: 0.9359 - val_loss: 1.6295 - val_accuracy: 0.7762
Epoch 7/20
79/78 [==============================] - 44s 560ms/step - loss: 0.2848 - accuracy: 0.9433 - val_loss: 1.5120 - val_accuracy: 0.7915

C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2048, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1024, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(512, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1203, activation="softmax", kernel_initializer="glorot_normal")`


Epoch 1/20
79/78 [==============================] - 44s 560ms/step - loss: 2.8765 - accuracy: 0.6385 - val_loss: 6.6765 - val_accuracy: 0.0572
Epoch 2/20
79/78 [==============================] - 43s 550ms/step - loss: 0.8685 - accuracy: 0.8661 - val_loss: 4.9582 - val_accuracy: 0.3428
Epoch 3/20
79/78 [==============================] - 44s 554ms/step - loss: 0.5813 - accuracy: 0.8985 - val_loss: 2.2696 - val_accuracy: 0.6941
Epoch 4/20
79/78 [==============================] - 44s 552ms/step - loss: 0.4457 - accuracy: 0.9181 - val_loss: 2.0031 - val_accuracy: 0.7501
Epoch 5/20
79/78 [==============================] - 44s 560ms/step - loss: 0.3717 - accuracy: 0.9290 - val_loss: 1.6435 - val_accuracy: 0.7680
Epoch 6/20
79/78 [==============================] - 45s 568ms/step - loss: 0.3156 - accuracy: 0.9378 - val_loss: 1.8599 - val_accuracy: 0.7753
Epoch 7/20
79/78 [==============================] - 44s 558ms/step - loss: 0.2883 - accuracy: 0.9421 - val_loss: 1.6842 - val_accuracy: 0.7906

C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2048, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1024, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(512, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1203, activation="softmax", kernel_initializer="glorot_normal")`


Epoch 1/20
79/78 [==============================] - 46s 579ms/step - loss: 2.8799 - accuracy: 0.6396 - val_loss: 6.6253 - val_accuracy: 0.0933
Epoch 2/20
79/78 [==============================] - 44s 560ms/step - loss: 0.8592 - accuracy: 0.8674 - val_loss: 4.7942 - val_accuracy: 0.4675
Epoch 3/20
79/78 [==============================] - 44s 554ms/step - loss: 0.5835 - accuracy: 0.8991 - val_loss: 2.2858 - val_accuracy: 0.7104
Epoch 4/20
79/78 [==============================] - 44s 555ms/step - loss: 0.4444 - accuracy: 0.9180 - val_loss: 1.5755 - val_accuracy: 0.7828
Epoch 5/20
79/78 [==============================] - 44s 557ms/step - loss: 0.3671 - accuracy: 0.9298 - val_loss: 1.7920 - val_accuracy: 0.7786
Epoch 6/20
79/78 [==============================] - 44s 556ms/step - loss: 0.3248 - accuracy: 0.9361 - val_loss: 1.4510 - val_accuracy: 0.8023
Epoch 7/20
79/78 [==============================] - 44s 559ms/step - loss: 0.2784 - accuracy: 0.9436 - val_loss: 1.6804 - val_accuracy: 0.7888

In [5]:
name = main_name + '_2'
CheckAccuracy(name)

hep_LEC_2
Validation Accuracy
[0.7287499904632568, 0.7648375034332275, 0.8947250247001648, 0.9191250205039978, 0.9050124883651733]
0.8424900054931641


In [5]:
#Final Model
## 1
main_name = 'hep_LEC'
name = main_name + '_1'
Train(X, Y1, save_root, name, epoch = 20, batch_size = 4096, nodes = [2048, 1024, 512])

## 2
name = main_name + '_2'
Train(X, Y2, save_root, name, epoch = 20, batch_size = 4096, nodes = [2048, 1024, 512])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2048, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1024, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(512, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1415, activation="softmax", kernel_initializer="glorot_normal")`



Epoch 1/20
98/97 [==============================] - 39s 393ms/step - loss: 0.5420 - accuracy: 0.9510
Epoch 2/20


C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\callbacks\callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


98/97 [==============================] - 38s 386ms/step - loss: 0.0313 - accuracy: 0.9946
Epoch 3/20
98/97 [==============================] - 38s 388ms/step - loss: 0.0224 - accuracy: 0.9959
Epoch 4/20
98/97 [==============================] - 39s 395ms/step - loss: 0.0168 - accuracy: 0.9967
Epoch 5/20
98/97 [==============================] - 38s 391ms/step - loss: 0.0142 - accuracy: 0.9972
Epoch 6/20
98/97 [==============================] - 39s 397ms/step - loss: 0.0115 - accuracy: 0.9977
Epoch 7/20
98/97 [==============================] - 39s 394ms/step - loss: 0.0094 - accuracy: 0.9981
Epoch 8/20
98/97 [==============================] - 39s 393ms/step - loss: 0.0084 - accuracy: 0.9982
Epoch 9/20
98/97 [==============================] - 39s 399ms/step - loss: 0.0075 - accuracy: 0.9984
Epoch 10/20
98/97 [==============================] - 40s 413ms/step - loss: 0.0073 - accuracy: 0.9985
Epoch 11/20
98/97 [==============================] - 40s 408ms/step - loss: 0.0063 - accuracy: 0.9987

C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2048, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1024, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(512, activation="relu", kernel_initializer="glorot_normal")`
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1203, activation="softmax", kernel_initializer="glorot_normal")`


Epoch 1/20
98/97 [==============================] - 38s 390ms/step - loss: 2.5599 - accuracy: 0.6771
Epoch 2/20


C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\callbacks\callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


98/97 [==============================] - 37s 375ms/step - loss: 0.7473 - accuracy: 0.8795
Epoch 3/20
98/97 [==============================] - 37s 376ms/step - loss: 0.5026 - accuracy: 0.9097
Epoch 4/20
98/97 [==============================] - 37s 381ms/step - loss: 0.3850 - accuracy: 0.9269
Epoch 5/20
98/97 [==============================] - 38s 385ms/step - loss: 0.3176 - accuracy: 0.9371
Epoch 6/20
98/97 [==============================] - 38s 388ms/step - loss: 0.2772 - accuracy: 0.9435
Epoch 7/20
98/97 [==============================] - 38s 383ms/step - loss: 0.2383 - accuracy: 0.9506
Epoch 8/20
98/97 [==============================] - 38s 386ms/step - loss: 0.2080 - accuracy: 0.9555
Epoch 9/20
98/97 [==============================] - 38s 388ms/step - loss: 0.1927 - accuracy: 0.9582
Epoch 10/20
98/97 [==============================] - 38s 388ms/step - loss: 0.1735 - accuracy: 0.9621
Epoch 11/20
98/97 [==============================] - 38s 386ms/step - loss: 0.1600 - accuracy: 0.9643

# LR

In [10]:
Y1_raw = ADoub_meta.iloc[:, 0].values.squeeze().astype(int)
Y2_raw = ADoub_meta.iloc[:, 2].values.squeeze().astype(int)

In [ ]:
kf = KFold(n_splits=5, random_state = 0, shuffle = True)
cv_index = kf.split(X)

Accuracy = []

i = 0

sta = time.time()

for cv_train, cv_test in cv_index:
    
    i += 1
    print(i)
    print('c1:')
    
    clf = LogisticRegression(n_jobs = 24).fit(X = X[cv_train, :],
                                              y = Y1_raw[cv_train])
    
    test_pred_c1 = clf.predict(X[cv_test, :])

    t1 = test_pred_c1 == Y1_raw[cv_test]
    c1_accuracy = t1.sum() / test_pred_c1.shape[0]
    
    print('c2:')
    clf = LogisticRegression(n_jobs = 24).fit(X = X[cv_train, :],
                                              y = Y2_raw[cv_train])
    
    test_pred_c2 = clf.predict(X[cv_test, :])

    t2 = test_pred_c2 == Y2_raw[cv_test]
    c2_accuracy = t2.sum() / test_pred_c2.shape[0]
    
    c12_accuracy = np.sum(t1&t2) / test_pred_c2.shape[0]
    
    print([c1_accuracy, c2_accuracy, c12_accuracy])
    Accuracy.append([c1_accuracy, c2_accuracy, c12_accuracy])

end = time.time()
dura = (end - sta)/3600
print(dura, 'h(s)')
print(np.mean(Accuracy, axis = 0))